In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction

### Add Directories/ Load Files

In [2]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

/Users/barbaramathiopoulou/OneDrive - Charité - Universitätsmedizin Berlin/ENTRAINMENT_PROJECT


In [124]:
fname = 'sub-014_ses-2021011402023495_run-BrainSense20210114020400.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'test',
        fname
    ),
    info = None
)

Creating RawArray with float64 data, n_channels=6, n_times=278125
    Range : 0 ... 278124 =      0.000 ...  1112.496 secs
Ready.


/opt/anaconda3/lib/python3.7/site-packages/mne/externals/pymatreader/utils.py:213: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
<ipython-input-124-26562827863d>:11: RuntimeWarning: Importing FieldTrip data without an info dict from the original file. Channel locations, orientations and types will be incorrect. The imported data cannot be used for source analysis, channel interpolation etc.
  info = None
<ipython-input-124-26562827863d>:11: RuntimeWarning: 4 channel names are too long, have been truncated to 15 characters:
['LFP_L_13_STN_PEAK13Hz_THR20-30_AVG3000ms', 'LFP_R_13_PEAK14Hz_THR20-30_AVG3000ms', 'STIM_L_130Hz_60us', 'STIM_R_130Hz_60us']
  info = None
<ipython-input-124-26562827863d>:11: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel mi

In [125]:
#Plot TF plot with artefact rejected data
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [126]:
fft_name = 'Sub0014_3mfu_FFt'

In [127]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'.pdf')
plt.savefig(str(fft_fig)+str(fft_name)+'.jpg')

np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [128]:
d = {'onset': [25, 907, 941],
    'duration': [30, 10, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

   onset  duration description
0     25        30     StimOff
1    907        10      StimOn
2    941        10     Rebound


In [131]:
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.2

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = 'Sub014_3mfu_LSTN_Contra'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.1, title1)

plt.subplot(1,2,2)
title2 = 'Sub014_3mfu_RSTN_Ipsi'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.4, title2)

plt.show()

In [132]:
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

     StimOff_LSTN  StimOn_LSTN  Rebound_LSTN  StimOff_RSTN  StimOn_RSTN  \
0        0.835734     0.119771      0.034390      2.691480     3.296489   
1        2.705534     0.548268      0.391605     10.953500    16.126455   
2        1.686843     0.588131      0.389346      9.426839    14.594199   
3        0.995324     0.355170      0.194296      6.330059     4.919660   
4        0.860407     0.251088      0.207339      5.196738     2.508055   
..            ...          ...           ...           ...          ...   
121      0.002774     0.003468      0.002906      0.025628    31.812602   
122      0.000248     0.000207      0.000247      0.000295     0.071230   
123      0.000246     0.000126      0.000286      0.000289     0.006608   
124      0.000233     0.000167      0.000168      0.000252     0.001918   
125      0.000118     0.000095      0.000059      0.000066     0.000662   

     Rebound_RSTN  
0        0.210361  
1        0.838321  
2        0.398880  
3        0.501498  

In [133]:
fft_name = 'Sub014_3mfu_PS'

In [134]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','test/')
fft_file = os.path.join(project_path, 'data','power_spectra','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'.pdf')
plt.savefig(str(fft_fig)+str(fft_name)+'.jpg')

ps_df.to_csv(str(fft_file)+str(fft_name)+ '.csv')